In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import pathlib
import numpy as np
import math
import glob
import os
%matplotlib inline
print(tf.__version__)

2.1.0


# config

In [2]:
EPOCHS = 10
BATCH_SIZE = 16
Width=224                                     #256
Hight=224
dataset_dir = "./data/dc_3000/"
train_dir = dataset_dir + "train"
valid_dir = dataset_dir + "val"
save_model_dir = "./weight/"

# data

In [3]:
train_image_path = glob.glob(train_dir+"/*/*.jpg")
valid_image_path = glob.glob(valid_dir+"/*/*.jpg")
# print(train_image_path)
# print(valid_image_path)
print(len(train_image_path))
print(len(valid_image_path))

3000
1300


In [4]:
def load_preprocess_image(path,label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image,channels=3)
    image = tf.image.resize(image,[Width,Hight])

    # 数据增强 -- 随机裁剪
    #image = tf.image.random_crop(image,[256,256,3])
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    # image = tf.image.random_brightness(image,0.5)
    # image = tf.image.random_contrast(image,0.3 ,1)

    image = tf.cast(image,tf.float32)
    image = image/255
    label = tf.reshape(label,[1])
    return image, label

In [5]:
def load_preprocess_valid_image(path,label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image,channels=3)
    image = tf.image.resize(image,[Width,Hight])
    image = tf.cast(image,tf.float32)
    image = image/255
    label = tf.reshape(label,[1])
    return image, label

### Make traindata && label

### train data

In [6]:
train_image_label = [int((p.split("\\")[1]))for p in train_image_path]    #only label
#train_image_label = [tf.one_hot(int((p.split("\\")[1])),3,on_value=1,off_value=None,axis=0)for p in train_image_path]  #one-hot
train_image_label = tf.cast(train_image_label,tf.float32)
print(train_image_label)


tf.Tensor([0. 0. 0. ... 2. 2. 2.], shape=(3000,), dtype=float32)


In [7]:
train_count = len(train_image_path)

train_image_ds = tf.data.Dataset.from_tensor_slices((train_image_path,train_image_label)) # 组合
train_image_ds = train_image_ds.map(load_preprocess_image)  # map
train_image_ds = train_image_ds.shuffle(train_count).batch(BATCH_SIZE)# shuffle && batch

In [8]:
train_image_ds

<BatchDataset shapes: ((None, 224, 224, 3), (None, 1)), types: (tf.float32, tf.float32)>

In [9]:
# for img,label in train_image_ds.take(1):
#     print(label)

### valid data

In [10]:
valid_image_label = [int((p.split("\\")[1]))for p in valid_image_path]    #only label
# valid_image_label = [tf.one_hot(int((p.split("\\")[1])),3,on_value=1,off_value=None,axis=0)for p in valid_image_path] #one-hot
valid_image_label = tf.cast(valid_image_label,tf.float32)
valid_count = len(valid_image_path)

valid_image_ds = tf.data.Dataset.from_tensor_slices((valid_image_path,valid_image_label))     #组合
valid_image_ds = valid_image_ds.map(load_preprocess_valid_image)#map
valid_image_ds = valid_image_ds.shuffle(valid_count).batch(BATCH_SIZE)    # shuffle && batch
valid_image_ds

<BatchDataset shapes: ((None, 224, 224, 3), (None, 1)), types: (tf.float32, tf.float32)>

In [11]:
# for img,label in valid_image_ds.take(1):
#     print(label)

### Model

In [12]:
# model = keras.Sequential([
#     tf.keras.layers.Conv2D(64, (3, 3), input_shape=(256, 256, 3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(),
#     tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(),
#     tf.keras.layers.GlobalAveragePooling2D(),
#     tf.keras.layers.Dense(128, activation='relu'),
#     tf.keras.layers.Dense(3),
#     tf.keras.layers.Activation('softmax', dtype='float32')
# ]) 


In [13]:
#[244,244]
# import resnet_model
# model = resnet_model.resnet50(
#         num_classes = 3,
#         use_l2_regularizer = True) 


In [26]:
from models.vgg16 import VGG16
model = VGG16(3)
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128)       0         
_________________________________________________________________
conv3_1 (Conv2D)             (None, 56, 56, 256)      

In [18]:
# #[244,244]
#from resnet_model3 import ResNet18
#model = ResNet18([2, 2, 2, 2])

# from models.resnet_model3 import resnet_18
# model=resnet_18()
# model.compile(optimizer='adam',
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
#               metrics=['sparse_categorical_accuracy'])

In [ ]:
# from models.resnet import resnet_18
# model = resnet_18()
# model.build(input_shape=(None, 224, 224, 3))
# model.summary()

### model test

In [19]:
imgs, labels = next(iter(train_image_ds))

In [20]:
pred = model(imgs)
print(pred.shape)
print(pred[0].numpy(),print(pred[0]))
print(labels[0].numpy(),print(labels[0]))
# print(np.array([p[0].numpy() for p in tf.cast(pred>0,tf.int32)]))
# print(np.array([l[0].numpy() for l in labels]))

(16, 3)
tf.Tensor([0.35845077 0.35123852 0.29031074], shape=(3,), dtype=float32)
[0.35845077 0.35123852 0.29031074] None
tf.Tensor([2.], shape=(1,), dtype=float32)
[2.] None


### train

In [27]:
# define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adadelta()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

valid_loss = tf.keras.metrics.Mean(name='valid_loss')
valid_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='valid_accuracy')

In [28]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(y_true=labels, y_pred=predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(grads_and_vars=zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

@tf.function
def valid_step(images, labels):
    predictions = model(images, training=False)
    v_loss = loss_object(labels, predictions)

    valid_loss(v_loss)
    valid_accuracy(labels, predictions)

In [ ]:
# optimizer = tf.keras.optimizers.Adadelta() #Adam()  SGD()
# loss_object =tf.keras.losses.SparseCategoricalCrossentropy() #BinaryCrossentropy()
# train_loss = tf.keras.metrics.Mean("train_loss")
# train_accuracy = tf.keras.metrics.SparseCategoricalCrossentropy() #Accuracy() CategoricalAccuracy() SparseCategoricalCrossentropy()
# valid_loss = tf.keras.metrics.Mean("valid_loss")
# valid_accuracy = tf.keras.metrics.SparseCategoricalCrossentropy() #Accuracy() CategoricalAccuracy() SparseCategoricalCrossentropy()

In [ ]:
# def train_step(images, labels):
#     with tf.GradientTape() as t:
#         pred = model(images,training=True)
#         #loss_step = tf.keras.losses.BinaryCrossentropy(from_logits=True)(labels, pred)
#         loss_step = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)(labels, pred)  # 这里参数的意思是没有激活
#     grads = t.gradient(loss_step, model.trainable_variables)
#     optimizer.apply_gradients(zip(grads, model.trainable_variables))
#     train_loss(loss_step)
#     train_accuracy(labels,pred)
#     #print(labels)
#     #print(pred)

# def valid_step(images, labels):
#     pred = model(images,training=False)
#     #loss_step = tf.keras.losses.BinaryCrossentropy(from_logits=True)(labels, pred)  # 这里参数的意思是没有激活
#     loss_step = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)(labels, pred)  # 这里参数的意思是没有激活
#     valid_loss(loss_step)
#     valid_accuracy(labels,pred)
#     #print(labels)
#     #print(pred)

In [29]:
# 定义四个个空列表进行每一个epoch的记录
EPOCHS=1000
train_loss_result = []
train_acc_result = []
valid_loss_result = []
valid_acc_result = []
for epoch in range(EPOCHS):

    for imgs_,labels_ in train_image_ds:
        train_step(imgs_, labels_)
    

    train_loss_result.append(train_loss.result())
    train_acc_result.append(train_accuracy.result())
    
    for imgs_,labels_ in valid_image_ds:
        valid_step(imgs_, labels_)


    valid_loss_result.append(valid_loss.result())
    valid_acc_result.append(valid_accuracy.result())

    print(f'Train Epoch:{epoch+1};loss{train_loss.result():.3f},accuracy:{train_accuracy.result():.3f},valid_loss:{valid_loss.result():.3f},valid_accuracy:{valid_accuracy.result():.3f};')

    train_loss.reset_states()
    valid_accuracy.reset_states()
    valid_loss.reset_states()
    valid_accuracy.reset_states()

Train Epoch:1;loss1.099,accuracy:0.370,valid_loss:1.098,valid_accuracy:0.454;
Train Epoch:2;loss1.098,accuracy:0.385,valid_loss:1.098,valid_accuracy:0.446;
Train Epoch:3;loss1.098,accuracy:0.400,valid_loss:1.098,valid_accuracy:0.451;
Train Epoch:4;loss1.098,accuracy:0.407,valid_loss:1.098,valid_accuracy:0.458;
Train Epoch:5;loss1.098,accuracy:0.423,valid_loss:1.098,valid_accuracy:0.459;
Train Epoch:6;loss1.098,accuracy:0.426,valid_loss:1.098,valid_accuracy:0.478;
Train Epoch:7;loss1.098,accuracy:0.434,valid_loss:1.098,valid_accuracy:0.488;
Train Epoch:8;loss1.098,accuracy:0.441,valid_loss:1.098,valid_accuracy:0.495;
Train Epoch:9;loss1.098,accuracy:0.445,valid_loss:1.098,valid_accuracy:0.506;
Train Epoch:10;loss1.098,accuracy:0.451,valid_loss:1.098,valid_accuracy:0.508;
Train Epoch:11;loss1.097,accuracy:0.455,valid_loss:1.097,valid_accuracy:0.538;


KeyboardInterrupt: 

In [ ]:
# start training
EPOCHS=1000
for epoch in range(EPOCHS):
    train_loss.reset_states()
    train_accuracy.reset_states()
    valid_loss.reset_states()
    valid_accuracy.reset_states()
    step = 0
    for images, labels in train_image_ds:
        step += 1
        train_step(images, labels)
        print("Epoch: {}/{}, step: {}/{}, loss: {:.5f}, accuracy: {:.5f}".format(epoch + 1,
                                                                                 EPOCHS,
                                                                                 step,
                                                                                 math.ceil(train_count / BATCH_SIZE),
                                                                                 train_loss.result(),
                                                                                 train_accuracy.result()))

    for valid_images, valid_labels in valid_image_ds:
        valid_step(valid_images, valid_labels)

    print("Epoch: {}/{}, train loss: {:.5f}, train accuracy: {:.5f}, "
          "valid loss: {:.5f}, valid accuracy: {:.5f}".format(epoch + 1,
                                                              EPOCHS,
                                                              train_loss.result(),
                                                              train_accuracy.result(),
                                                              valid_loss.result(),
                                                              valid_accuracy.result()))




In [30]:
# model.save_weights(filepath=save_model_dir, save_format='tf')

model_path = save_model_dir+"/fashion_mnist.h5"
print(model_path)
model.save(model_path)


./weight//fashion_mnist.h5
